In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LinearSegmentedColormap

plt.rcParams.update({'font.size': 12})
plt.rcParams.update({'axes.labelsize': 12})

In [ ]:
def plot_antrag_subplots(vals_to_plot_dwd_netatmo,
                         vals_to_plot_dwd,
                         vals_to_plot_netatmo,
                         out_plot_path,
                         temp_agg,
                         event_date,
                         radar_data,
                         radar_lon,
                         radar_lat,
                         save_acc=''):
    '''plot interpolated events, grid wise '''

    if temp_agg == '60min':
        min_val, max_val = 0, 30
        clbr_label = 'mm/h'  # 'Hourly precipitation [m]'
        bound_ppt = [0., 1, 2, 4, 8, 10, 15, 20, 25, 30]  # , 40, 45]
    if temp_agg == '1440min':
        min_val, max_val = 0, 45
        clbr_label = 'mm/d'
        bound_ppt = [0., 1, 2, 4, 8, 10, 15, 20, 25, 30, 40, 45]

    plt.ioff()

    # cmap ppt
    interval_ppt = np.linspace(0.05, 0.95)
    colors_ppt = plt.get_cmap('jet_r')(interval_ppt)
    cmap_ppt = LinearSegmentedColormap.from_list('name', colors_ppt)

    cmap_ppt.set_over('navy')
    norm_ppt = mcolors.BoundaryNorm(bound_ppt, cmap_ppt.N)
    
    # cmapp fidd map
    bound_diff = [-15, -10, -5, -2, -1, 0, 1, 2, 5, 10, 15]
    _fontsize = 8
    color_fontsize = 12
    # Remove the middle 10% of the RdBu_r colormap
    interval = np.hstack([np.linspace(0, 0.3), np.linspace(0.7, 1)])
    colors = plt.get_cmap('PiYG')(interval)
    cmap_diff = LinearSegmentedColormap.from_list('name', colors)

    cmap_diff.set_over('darkslategrey')  # darkslategrey
    cmap_diff.set_under('crimson')

    norm_diff = mcolors.BoundaryNorm(bound_diff, cmap_diff.N)

    fig = plt.figure(figsize=(12, 12), constrained_layout=False, dpi=600)
    gs = gridspec.GridSpec(2, 3, width_ratios=[1, 1, 1, 1, 1, 1])


    # dwd+netatmo
    ax1 = fig.add_subplot(gs[:1, :2])
    ax1.scatter(x_coords_grd, y_coords_grd,
                c=vals_to_plot_dwd,
                marker=',', s=40, cmap=cmap_ppt,
                vmin=min_val,
                norm=norm_ppt,
                vmax=max_val)

    ax1.legend(title='DWD (a)', loc='upper left',  # upper
               frameon=False, fontsize=_fontsize)._legend_box.align = 'left'
    # dwd
    ax2 = fig.add_subplot(gs[:1, 2:4])
    im2 = ax2.scatter(x_coords_grd, y_coords_grd,
                      c=vals_to_plot_netatmo,
                      marker=',', s=40, cmap=cmap_ppt,
                      vmin=min_val,
                      norm=norm_ppt,
                      vmax=max_val)
#     ax2.scatter(dwd_xcoords, dwd_ycoords, c='darkgreen',
#                 marker='x', s=10, alpha=0.25)
    ax2.legend(title='Netatmo (b)', loc='upper left',
               frameon=False, fontsize=_fontsize)._legend_box.align = 'left'

    # colorbar
    cax0 = fig.add_subplot(gs[:1, 4:5])

    divider0 = make_axes_locatable(cax0)
    cax20 = divider0.append_axes("left", size="8%", pad=0.00001)

    cb0 = fig.colorbar(im2, ax=ax2, cax=cax20, norm=norm_ppt,
                       ticks=bound_ppt, label=clbr_label,
                       extend='max')

    cb0.set_ticks(bound_ppt)
    cb0.ax.tick_params(labelsize=color_fontsize)
    #==========================================================================
    # # second row
    #==========================================================================

    # dwd_netatmo
    ax5 = fig.add_subplot(gs[1:, :2])
    ax5.scatter(x_coords_grd, y_coords_grd,
                c=vals_to_plot_dwd_netatmo,
                marker=',', s=30, cmap=cmap_ppt,
                vmin=min_val, norm=norm_ppt, vmax=max_val)
    ax5.legend(title='DWD + Netatmo (c)', loc='upper left',
               frameon=False, fontsize=_fontsize)._legend_box.align = 'left'

    # radolan
    ax_rad = fig.add_subplot(gs[1:, 2:4])

    im_rad = ax_rad.scatter(radar_lon, radar_lat,
                            c=radar_data, cmap=cmap_ppt, s=30, marker=',',
                            vmin=min_val, norm=norm_ppt, vmax=max_val)
    ax_rad.legend(title='Radolan (c)', loc='upper left',
                  frameon=False, fontsize=_fontsize)._legend_box.align = 'left'

    ###
    cax = fig.add_subplot(gs[1:, 4:5])

    divider = make_axes_locatable(cax)
    cax2 = divider.append_axes("left", size="8%", pad=0.00001)

    cb1 = fig.colorbar(im_rad, ax=ax_rad, cax=cax2, norm=norm_diff,
                       ticks=bound_diff, label=clbr_label,
                       extend='both')
    cb1.set_ticks(bound_diff)
    cb1.ax.tick_params(labelsize=color_fontsize)
    plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])

    ax1.axis('off'), ax2.axis('off')
    ax5.axis('off'), ax_rad.axis('off')
    cax.axis('off'), cax0.axis('off')

    # plt.show()
    plt.savefig((
        out_plot_path / (
                '%s_%s_%s_event_test' %
                (save_acc, temp_agg,
                 str(event_date).replace(
                     '-', '_').replace(':',
                                       '_').replace(' ', '_')))),
                papertype='a4',
                bbox_inches='tight',
                pad_inches=0.05)
    plt.close()

    pass


#==============================================================================
